In [1]:
from pyspark.sql import SparkSession
import numpy as np
import os

spark = SparkSession.builder.appName('linreg2').getOrCreate()
print(dir(spark))

['Builder', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_activeSession', '_convert_from_pandas', '_createFromLocal', '_createFromRDD', '_create_dataframe', '_create_from_pandas_with_arrow', '_create_shell_session', '_get_numpy_record_dtype', '_inferSchema', '_inferSchemaFromList', '_instantiatedSession', '_jsc', '_jsparkSession', '_jvm', '_jwrapped', '_repr_html_', '_sc', '_wrapped', 'builder', 'catalog', 'conf', 'createDataFrame', 'getActiveSession', 'newSession', 'range', 'read', 'readStream', 'sparkContext', 'sql', 'stop', 'streams', 'table', 'udf', 'version']


In [27]:
# function imports
from pyspark.sql.functions import isnan,count,when
from pyspark.sql.functions import col as spark_col

In [2]:
data_path = '../data/cruise_ship_info.csv'
df_orig = spark.read.csv(data_path,inferSchema = True,header=True)
df_orig.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [3]:
print('Data Shape : ',(df_orig.count(),len(df_orig.columns))) # get shape 
data = df_orig.select('*') # create copy of data

Data Shape :  (158, 9)


In [5]:
for i in range(5):
    print(data.head(5)[i].asDict()) # data example

{'Ship_name': 'Journey', 'Cruise_line': 'Azamara', 'Age': 6, 'Tonnage': 30.276999999999997, 'passengers': 6.94, 'length': 5.94, 'cabins': 3.55, 'passenger_density': 42.64, 'crew': 3.55}
{'Ship_name': 'Quest', 'Cruise_line': 'Azamara', 'Age': 6, 'Tonnage': 30.276999999999997, 'passengers': 6.94, 'length': 5.94, 'cabins': 3.55, 'passenger_density': 42.64, 'crew': 3.55}
{'Ship_name': 'Celebration', 'Cruise_line': 'Carnival', 'Age': 26, 'Tonnage': 47.262, 'passengers': 14.86, 'length': 7.22, 'cabins': 7.43, 'passenger_density': 31.8, 'crew': 6.7}
{'Ship_name': 'Conquest', 'Cruise_line': 'Carnival', 'Age': 11, 'Tonnage': 110.0, 'passengers': 29.74, 'length': 9.53, 'cabins': 14.88, 'passenger_density': 36.99, 'crew': 19.1}
{'Ship_name': 'Destiny', 'Cruise_line': 'Carnival', 'Age': 17, 'Tonnage': 101.353, 'passengers': 26.42, 'length': 8.92, 'cabins': 13.21, 'passenger_density': 38.36, 'crew': 10.0}


In [30]:
###  check for null values

# Method 1

# def get_null_count(data,col):
#     return (data.count() - data.select(col).na.drop(how="any").count())

# for col in data.columns:
#     print('{} : {}'.format(col,get_null_count(data,col)))

# Method 2


data.select([count(when( isnan(c) | spark_col(c).isNull(),c)).alias(c) for c in data.columns]).show()

+---------+-----------+---+-------+----------+------+------+-----------------+----+
|Ship_name|Cruise_line|Age|Tonnage|passengers|length|cabins|passenger_density|crew|
+---------+-----------+---+-------+----------+------+------+-----------------+----+
|        0|          0|  0|      0|         0|     0|     0|                0|   0|
+---------+-----------+---+-------+----------+------+------+-----------------+----+



In [24]:
data.select([count(when(isnan(c),c)).alias(c) for c in data.columns]).show()

+---------+-----------+---+-------+----------+------+------+-----------------+----+
|Ship_name|Cruise_line|Age|Tonnage|passengers|length|cabins|passenger_density|crew|
+---------+-----------+---+-------+----------+------+------+-----------------+----+
|        0|          0|  0|      0|         0|     0|     0|                0|   0|
+---------+-----------+---+-------+----------+------+------+-----------------+----+



'crew'